In [1]:
import datajoint as dj
import numpy as np
import time
from tqdm import tqdm

In [2]:
#setting the address and the username
dj.config['database.host'] = '10.28.0.34'
dj.config['database.user'] = 'celiib'
dj.config['database.password'] = 'newceliipass'
dj.config['safemode']=True
dj.config["display.limit"] = 20

pinky = dj.create_virtual_module('pinky', 'microns_pinky')
schema = dj.schema('microns_pinky')


Connecting celiib@10.28.0.34:3306


In [3]:
@schema
class CoarseLabelFinal(dj.Computed):
    definition = """
    # Ensure that Vertex labels for pinky.ProofreadLabel match the triangle labels, so these are regenerated from the correct triangle labels.
    -> pinky.ProofreadLabel
    ---
    vertices  : longblob # Corrected vertex labels
    triangles : longblob # Same triangle labels as pinky.ProofreadLabel
    """
    
    key_source = pinky.ProofreadLabel & 'status="complete"'
    
    def make(self, key):
        start = time.time()
        
        print(key["segment_id"])
        labels = (pinky.ProofreadLabel & key).fetch1()
        corrected_vertex_labels = np.zeros(labels['vertices'].shape, np.uint8)
        
        mesh = (pinky.PymeshfixDecimatedExcitatoryStitchedMesh & 'decimation_ratio=0.35' & dict(segment_id=key['segment_id'])).fetch1()
        mesh_triangles = mesh['triangles']
        
        vertex_label_dict = dict()
        for i, label in enumerate(labels['triangles']):
            triangle = mesh_triangles[i]
            for node in triangle:
                if node in vertex_label_dict:
                    if vertex_label_dict[node] < label:
                        vertex_label_dict[node] = label
                else:
                    vertex_label_dict[node] = label
                
        for node, label in vertex_label_dict.items():
            corrected_vertex_labels[node] = label
            
        self.insert1(dict(key,
                          vertices=corrected_vertex_labels,
                          triangles=labels['triangles']))
        
        print("Segment {} vertex labels regenerated in: {} seconds.".format(key['segment_id'], time.time() - start))

In [4]:
CoarseLabelFinal()

segmentation segmentation id,segment_id segment id unique within each Segmentation,decimation_ratio,author_original name of last editor,author_proofreader name of last editor,date_time the last time it was edited,vertices Corrected vertex labels,triangles Same triangle labels as pinky.ProofreadLabel
3,648518346341371119,0.35,computer_Auto,celiib,2019-07-12 11:52:31,=BLOB=,=BLOB=
3,648518346349386137,0.35,computer_Auto,celiib,2019-06-11 18:53:38,=BLOB=,=BLOB=
3,648518346349470171,0.35,computer_Auto,celiib,2019-06-11 18:49:31,=BLOB=,=BLOB=
3,648518346349471156,0.35,computer_Auto,celiib,2019-06-12 13:53:20,=BLOB=,=BLOB=
3,648518346349471500,0.35,computer_Auto,celiib,2019-06-12 14:18:35,=BLOB=,=BLOB=
3,648518346349471562,0.35,computer_Auto,celiib,2019-07-12 11:47:49,=BLOB=,=BLOB=
3,648518346349471565,0.35,computer_Auto,celiib,2019-06-12 14:11:11,=BLOB=,=BLOB=
3,648518346349471910,0.35,computer_Auto,celiib,2019-07-12 11:44:28,=BLOB=,=BLOB=
3,648518346349472574,0.35,computer_Auto,celiib,2019-06-12 14:27:07,=BLOB=,=BLOB=
3,648518346349472601,0.35,computer_Auto,celiib,2019-07-12 11:42:05,=BLOB=,=BLOB=


In [5]:
start = time.time()
CoarseLabelFinal.populate(reserve_jobs=True)
print(time.time() - start)

648518346349473597
Segment 648518346349473597 vertex labels regenerated in: 3.577993392944336 seconds.
3.850841522216797


In [6]:
#(schema.jobs & "table_name='__coarse_label_final'").delete()